In [64]:
import pandas as pd
import math
from scipy.stats import norm
import numpy as np
import itertools
from ast import literal_eval

In [65]:
df = pd.read_csv("./data/info/pad_fixed.csv",)
t_df = pd.read_json("./data/info/train_pairs.json", orient="records")
g_scores = np.array(t_df[t_df.type=="genuine"].match_score)
i_scores = np.array(t_df[t_df.type=="impostor"].match_score)

In [66]:
def f_prob(scores):
  mean, std = norm.fit(scores)
  f_norm = norm(mean, std)
  var_p = .5
  def inner(score):
    return f_norm.pdf(score) * var_p
  return inner

In [67]:
f_g = f_prob(g_scores)
f_i = f_prob(i_scores)
def f_prop_of_impostor(score):
  f_i_res = f_i(score)
  return f_i_res / (f_i_res + f_g(score))
log_of_prop = lambda x:  math.log(f_prop_of_impostor(x))

In [68]:
df["train_match_scores"] = df.train_match_scores.apply(literal_eval)

In [69]:
scores_list_of_list = df["train_match_scores"].to_numpy()
merged = list(itertools.chain.from_iterable(scores_list_of_list))
unique_values = list(set(merged))
hash_table = {}
for value in unique_values:
    hash_table[value] = math.log(f_prop_of_impostor(value))

In [70]:
def calculate_props(match_scores):
  match_scores = np.array(match_scores)
  props = [hash_table[score] for score in match_scores]
  return np.sum(props)

In [71]:
for index, row in df.iterrows():
    df.loc[index, 'prop_of_i'] = calculate_props(row.train_match_scores)

In [72]:
for i in range(1,6):
  fold_df = df[df["fold"] == i]
  fold_df = fold_df[fold_df["pad_status"] == True]
  fold_df = fold_df.sort_values(by="prop_of_i").head(400)
  for idx, row in fold_df.iterrows():
    df.loc[idx, "verieye_status"] = True

In [73]:
# df.to_json("synthetic.json", orient="records")